 <center>JCAFB-2025 - Cadastro de Pacientes Off-Line<center>
 <center>Campanha DHC (1) - 2025<center>

# Exportar a lista de todos os Pacientes cadastrados (Referência)

1. Conectar-se, via *browser*, ao *Odoo* do servidor [clvheatlh-jcafb-2025n-aws-pro](https://34.224.24.169).

1. Acessar a *View* *Patients*:
    - Menu de acesso:
        - **Health** » **Health** » **Patient** » **Patients**

1. Selecionar todos os Pacientes.

1. Executar a Ação "**Exportar**" (**843**):
    - Parâmetros utilizados:
        - *Quero atualizar os dados (compatibilidade com importação e exportação)*: "**marcado**"
        - *Campos para exportar*: » *Modelo*: "**JCAFB-2025 - Cadastro de Pacientes (Ref)**"
    1. Utilize o botão **Exportar** para executar a Ação.


# Parâmetros globais

In [1]:
import pandas as pd
import re


In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)


## Cadastro de Referência

In [3]:
jcafb_cadastro_pacientes_ref_path = 'data/jcafb_2025_cadastro_pacientes_dhc1_ref.xlsx'
jcafb_cadastro_pacientes_ref_sheet = 'cadastro_pacientes_dhc1_ref'

## Cadastros de Campanha

### Cadastro de Campanha (arquivo 1)

In [4]:
jcafb_cadastro_campanha_1_path = 'data/jcafb_2025_cadastro_pacientes_dhc1_a1.xlsx'
jcafb_cadastro_campanha_1_sheet = 'cadastro_pacientes_dhc1_a1'

### Cadastro de Campanha (arquivo 2)

In [5]:
jcafb_cadastro_campanha_2_path = 'data/jcafb_2025_cadastro_pacientes_dhc1_a2.xlsx'
jcafb_cadastro_campanha_2_sheet = 'cadastro_pacientes_dhc1_a2'

### Cadastro de Campanha (arquivo 3)

In [6]:
jcafb_cadastro_campanha_3_path = 'data/jcafb_2025_cadastro_pacientes_dhc1_a3.xlsx'
jcafb_cadastro_campanha_3_sheet = 'cadastro_pacientes_dhc1_a3'

### Cadastro de Campanha (arquivo 4)

In [7]:
jcafb_cadastro_campanha_4_path = 'data/jcafb_2025_cadastro_pacientes_dhc1_a4.xlsx'
jcafb_cadastro_campanha_4_sheet = 'cadastro_pacientes_dhc1_a4'

## Arquivos de Saída

### Lista de Códigos Inválidos

In [8]:
invalid_codes_path = 'data/jcafb_2025_dhc1_invalid_codes.xlsx'
invalid_codes_sheet = 'dhc1_invalid_codes'

### Lista de Ids incluídos

In [9]:
new_ids_path = 'data/jcafb_2025_dhc1_new_ids.xlsx'
new_ids_sheet = 'dhc1_new_ids'

### Lista de Registros incluídos

In [10]:
new_records_path = 'data/jcafb_2025_dhc1_new_records.xlsx'
new_records_sheet = 'dhc1_new_records'

### Lista de Atualizações

In [11]:
updt_records_path = 'data/jcafb_2025_dhc1_updt_records.xlsx'
updt_records_sheet = 'dhc1_updt_records'

### Cadastro de Pacientes Atualizado

In [12]:
jcafb_cadastro_pacientes_updt_path = 'data/jcafb_2025_cadastro_pacientes_dhc1_updt.xlsx'
jcafb_cadastro_pacientes_updt_sheet = 'cadastro_pacientes_dhc1_updt'

# Funções de Suporte

In [13]:
def somaPoderada(numero):
    i = 0
    soma = 0
    while i<len(numero):
        soma = soma+int(numero[i]) * (15 - i)
        i = i+1
    return soma

def validaCNS(numero):
    numero = str(numero)
    if numero.isdigit():
        if re.match(r'[1-2]\d{10}00[0-1]\d$',numero) or re.match(r'[7-9]\d{14}$',numero):
            return somaPoderada(numero) % 11 == 0
    return False

# validaCNS('703008820145471')
# validaCNS('702803658947067')
# validaCNS('704301554368493')
# validaCNS('700506784991850')


In [14]:
def format_code(code_seq):
    code = list(map(int, str(code_seq)))
    code_len = len(code)
    while len(code) < 14:
        code.insert(0, 0)
    while len(code) < 16:
        n = sum([(len(code) + 1 - i) * v for i, v in enumerate(code)]) % 11
        if n > 1:
            f = 11 - n
        else:
            f = 0
        code.append(f)
    code_str = "%s.%s.%s.%s.%s-%s" % (str(code[0]) + str(code[1]),
                                      str(code[2]) + str(code[3]) + str(code[4]),
                                      str(code[5]) + str(code[6]) + str(code[7]),
                                      str(code[8]) + str(code[9]) + str(code[10]),
                                      str(code[11]) + str(code[12]) + str(code[13]),
                                      str(code[14]) + str(code[15]))
    if code_len <= 3:
        code_form = code_str[18 - code_len:21]
    elif code_len > 3 and code_len <= 6:
        code_form = code_str[17 - code_len:21]
    elif code_len > 6 and code_len <= 9:
        code_form = code_str[16 - code_len:21]
    elif code_len > 9 and code_len <= 12:
        code_form = code_str[15 - code_len:21]
    elif code_len > 12 and code_len <= 14:
        code_form = code_str[14 - code_len:21]
    return code_form

def validaCPF(code):

    if code is not False:

        sequence_str = ''
        for c in code:
            if c.isdigit():
                sequence_str = sequence_str + c
        sequence_str = sequence_str[:len(sequence_str) - 2]
        _format_code = format_code(sequence_str)

        if code == _format_code:
            return True

    return False

# validaCPF('551.583.048-23')
# validaCPF('525.338.988-13')

# Leitura do Cadastro de Referência

In [15]:
xls_jcafb_cadastro_pacientes_ref = pd.ExcelFile(jcafb_cadastro_pacientes_ref_path)
# jcafb_cadastro_pacientes_ref = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_pacientes_ref_sheet, index_col=0, na_filter=False)
jcafb_cadastro_pacientes_ref = pd.read_excel(xls_jcafb_cadastro_pacientes_ref,
                                             sheet_name=jcafb_cadastro_pacientes_ref_sheet, na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Columns:', jcafb_cadastro_pacientes_ref.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Number of rows:', len(jcafb_cadastro_pacientes_ref))


INFO:__main__:--> jcafb_cadastro_pacientes_ref Columns: ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'reg_state', 'employee_id/name', 'employee_id/id', 'category_names', 'category_ids/id', 'marker_names', 'marker_ids/id', 'tag_names', 'tag_ids/id', 'verification_state', 'verification_outcome_infos']
INFO:__main__:--> jcafb_cadastro_pacientes_ref Number of rows: 843


# Leitura do Cadastro Atualizado

In [16]:
xls_jcafb_cadastro_pacientes_updt = pd.ExcelFile(jcafb_cadastro_pacientes_ref_path)
# jcafb_cadastro_pacientes_ref = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_pacientes_ref_sheet, index_col=0, na_filter=False)
jcafb_cadastro_pacientes_updt = pd.read_excel(xls_jcafb_cadastro_pacientes_updt, sheet_name=jcafb_cadastro_pacientes_ref_sheet, na_filter=False)

pos = len(jcafb_cadastro_pacientes_updt.columns)
jcafb_cadastro_pacientes_updt.insert(pos, 'revised', '')
jcafb_cadastro_pacientes_updt.insert(pos, 'new', '')

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Columns:', jcafb_cadastro_pacientes_updt.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Number of rows:', len(jcafb_cadastro_pacientes_updt))


INFO:__main__:--> jcafb_cadastro_pacientes_ref Columns: ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'reg_state', 'employee_id/name', 'employee_id/id', 'category_names', 'category_ids/id', 'marker_names', 'marker_ids/id', 'tag_names', 'tag_ids/id', 'verification_state', 'verification_outcome_infos', 'new', 'revised']
INFO:__main__:--> jcafb_cadastro_pacientes_ref Number of rows: 843


# Leitura dos Cadastros de Campanha

## Cadastro de Campanha (arquivo 1)

In [17]:
xls_jcafb_cadastro_campanha_1 = pd.ExcelFile(jcafb_cadastro_campanha_1_path)
jcafb_cadastro_campanha_1 = pd.read_excel(xls_jcafb_cadastro_campanha_1, sheet_name=jcafb_cadastro_campanha_1_sheet, na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_1 Columns:', jcafb_cadastro_campanha_1.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_1 Number of rows:', len(jcafb_cadastro_campanha_1))


INFO:__main__:--> jcafb_cadastro_campanha_1 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'reg_state', 'employee_id/name', 'employee_id/id', 'category_names', 'category_ids/id', 'marker_names', 'marker_ids/id', 'tag_names', 'tag_ids/id', 'verification_state', 'verification_outcome_infos']
INFO:__main__:--> jcafb_cadastro_campanha_1 Number of rows: 844


## Cadastro de Campanha (arquivo 2)

In [18]:
xls_jcafb_cadastro_campanha_2 = pd.ExcelFile(jcafb_cadastro_campanha_2_path)
jcafb_cadastro_campanha_2 = pd.read_excel(xls_jcafb_cadastro_campanha_2, sheet_name=jcafb_cadastro_campanha_2_sheet, na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_2 Columns:', jcafb_cadastro_campanha_2.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_2 Number of rows:', len(jcafb_cadastro_campanha_2))


INFO:__main__:--> jcafb_cadastro_campanha_2 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'reg_state', 'employee_id/name', 'employee_id/id', 'category_names', 'category_ids/id', 'marker_names', 'marker_ids/id', 'tag_names', 'tag_ids/id', 'verification_state', 'verification_outcome_infos']
INFO:__main__:--> jcafb_cadastro_campanha_2 Number of rows: 844


## Cadastro de Campanha (arquivo 3)

In [19]:
xls_jcafb_cadastro_campanha_3 = pd.ExcelFile(jcafb_cadastro_campanha_3_path)
jcafb_cadastro_campanha_3 = pd.read_excel(xls_jcafb_cadastro_campanha_3, sheet_name=jcafb_cadastro_campanha_3_sheet, na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_3 Columns:', jcafb_cadastro_campanha_3.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_3 Number of rows:', len(jcafb_cadastro_campanha_3))


INFO:__main__:--> jcafb_cadastro_campanha_3 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'reg_state', 'employee_id/name', 'employee_id/id', 'category_names', 'category_ids/id', 'marker_names', 'marker_ids/id', 'tag_names', 'tag_ids/id', 'verification_state', 'verification_outcome_infos']
INFO:__main__:--> jcafb_cadastro_campanha_3 Number of rows: 844


## Cadastro de Campanha (arquivo 4)

In [20]:
xls_jcafb_cadastro_campanha_4 = pd.ExcelFile(jcafb_cadastro_campanha_4_path)
jcafb_cadastro_campanha_4 = pd.read_excel(xls_jcafb_cadastro_campanha_4, sheet_name=jcafb_cadastro_campanha_4_sheet, na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_4 Columns:', jcafb_cadastro_campanha_4.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_4 Number of rows:', len(jcafb_cadastro_campanha_4))


INFO:__main__:--> jcafb_cadastro_campanha_4 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'reg_state', 'employee_id/name', 'employee_id/id', 'category_names', 'category_ids/id', 'marker_names', 'marker_ids/id', 'tag_names', 'tag_ids/id', 'verification_state', 'verification_outcome_infos']
INFO:__main__:--> jcafb_cadastro_campanha_4 Number of rows: 844


# Validar Cadastros de Campanha

## Validação do "Código do Paciente"

In [21]:
column_names = ['íd', 'name', 'code', 'source', 'row']
invalid_codes = pd.DataFrame(columns=column_names)

def code_validation(jcafb_cadastro_campanha, source):

    for i, row in jcafb_cadastro_campanha.iterrows():

        _code = row['code']
        validate = False
        if validaCNS(_code):
            validate = True
        if not validate:
            if validaCPF(_code):
                validate = True
        if not validate:
            new_row = {'id': row['id'],
                       'name': row['name'],
                       'code': _code,
                       'row': i + 2,
                       'source': source
                      }
            _logger.info(u'%s %s %s', '-->', 'Invalid Code:', new_row)
            invalid_codes.loc[len(invalid_codes)] = new_row


### Cadastro de Campanha (arquivo 1)

In [22]:
code_validation(jcafb_cadastro_campanha_1, 'arquivo_1')


INFO:__main__:--> Invalid Code: {'id': '', 'name': 'Carlos Eduardo Vercelino', 'code': '036.731.518-17', 'row': 845, 'source': 'arquivo_1'}


### Cadastro de Campanha (arquivo 2)

In [23]:
code_validation(jcafb_cadastro_campanha_2, 'arquivo_2')


INFO:__main__:--> Invalid Code: {'id': '', 'name': 'Lamara Guiçardi Vercelino', 'code': '158.522.278-02', 'row': 845, 'source': 'arquivo_2'}


### Cadastro de Campanha (arquivo 3)

In [24]:
code_validation(jcafb_cadastro_campanha_3, 'arquivo_3')


INFO:__main__:--> Invalid Code: {'id': '', 'name': 'Maria Nilde Damico Guicardi', 'code': '106.499.678-78', 'row': 845, 'source': 'arquivo_3'}


### Cadastro de Campanha (arquivo 4)

In [25]:
code_validation(jcafb_cadastro_campanha_4, 'arquivo_4')


INFO:__main__:--> Invalid Code: {'id': '', 'name': 'Moacyr Guicardi', 'code': '306.986.268-33', 'row': 845, 'source': 'arquivo_4'}


## Lista de Códigos Inválidos

In [26]:
invalid_codes

,íd,name,code,source,row
0,NaN,Carlos Eduardo Vercelino,036.731.518-17,arquivo_1,845
1,NaN,Lamara Guiçardi Vercelino,158.522.278-02,arquivo_2,845
2,NaN,Maria Nilde Damico Guicardi,106.499.678-78,arquivo_3,845
3,NaN,Moacyr Guicardi,306.986.268-33,arquivo_4,845


## Inclusão de novos "ids"

In [27]:
column_names = ['id', 'name', 'code', 'source', 'row']
new_ids = pd.DataFrame(columns=column_names)

def new_ids_inclusion(jcafb_cadastro_campanha, source):

    for i, row in jcafb_cadastro_campanha.iterrows():

        _id = row['id']
        _code = row['code']

        if _id == '':
            sequence_str = ''
            for c in _code:
                if c.isdigit():
                    sequence_str = sequence_str + c
            _id = '__import__.clv_patient_' + sequence_str
            new_row = {'id': _id,
                       'name': row['name'],
                       'code': _code,
                       'source': source,
                       'row': i + 2
                      }
            _logger.info(u'%s %s %s', '-->', 'Novo id:', new_row)
            new_ids.loc[len(new_ids)] = new_row
            jcafb_cadastro_campanha['id'].values[i] = _id


### Cadastro de Campanha (arquivo 1)

In [28]:
new_ids_inclusion(jcafb_cadastro_campanha_1, 'arquivo_1')


INFO:__main__:--> Novo id: {'id': '__import__.clv_patient_03673151817', 'name': 'Carlos Eduardo Vercelino', 'code': '036.731.518-17', 'source': 'arquivo_1', 'row': 845}


### Cadastro de Campanha (arquivo 2)

In [29]:
new_ids_inclusion(jcafb_cadastro_campanha_2, 'arquivo_2')


INFO:__main__:--> Novo id: {'id': '__import__.clv_patient_15852227802', 'name': 'Lamara Guiçardi Vercelino', 'code': '158.522.278-02', 'source': 'arquivo_2', 'row': 845}


### Cadastro de Campanha (arquivo 3)

In [30]:
new_ids_inclusion(jcafb_cadastro_campanha_3, 'arquivo_3')


INFO:__main__:--> Novo id: {'id': '__import__.clv_patient_10649967878', 'name': 'Maria Nilde Damico Guicardi', 'code': '106.499.678-78', 'source': 'arquivo_3', 'row': 845}


### Cadastro de Campanha (arquivo 4)

In [31]:
new_ids_inclusion(jcafb_cadastro_campanha_4, 'arquivo_4')


INFO:__main__:--> Novo id: {'id': '__import__.clv_patient_30698626833', 'name': 'Moacyr Guicardi', 'code': '306.986.268-33', 'source': 'arquivo_4', 'row': 845}


## Lista de Ids incluídos

In [32]:
new_ids

,id,name,code,source,row
0,__import__.clv_patient_03673151817,Carlos Eduardo Vercelino,036.731.518-17,arquivo_1,845
1,__import__.clv_patient_15852227802,Lamara Guiçardi Vercelino,158.522.278-02,arquivo_2,845
2,__import__.clv_patient_10649967878,Maria Nilde Damico Guicardi,106.499.678-78,arquivo_3,845
3,__import__.clv_patient_30698626833,Moacyr Guicardi,306.986.268-33,arquivo_4,845


# Atualizar Cadastro

## Incluir novos registros

In [33]:
column_names = ['id', 'name', 'code', 'source', 'source_row', 'updt_row']
new_records = pd.DataFrame(columns=column_names)

def new_records_inclusion(jcafb_cadastro_campanha, source):

    for i, row in jcafb_cadastro_campanha.iterrows():

        _id = row['id']
        index = jcafb_cadastro_pacientes_ref.loc[jcafb_cadastro_pacientes_ref['id'] == _id]
        if len(index) == 0:
            # new_row = {'id': _id,
            #            'name': row['name'],
            #            'code': row['code'],
            #            'birthday': row['birthday'],
            #            'gender': row['gender'],
            #           }
            new_record_pos = len(jcafb_cadastro_pacientes_updt)
            # jcafb_cadastro_pacientes_updt.loc[new_record_pos] = new_row
            jcafb_cadastro_pacientes_updt.loc[new_record_pos] = row
            jcafb_cadastro_pacientes_updt['new'].values[new_record_pos] = 'x'
            _logger.info(u'%s %s %s', '-->', 'Novo Registro:', jcafb_cadastro_pacientes_updt.values[new_record_pos])

            new_row = {'id': _id,
                       'name': row['name'],
                       'code': row['code'],
                       'source': source,
                       'source_row': i + 2,
                       'updt_row': new_record_pos + 2
                      }
            pos = len(new_records)
            new_records.loc[pos] = new_row


### Cadastro de Campanha (arquivo 1)

In [34]:
new_records_inclusion(jcafb_cadastro_campanha_1, 'arquivo_1')


INFO:__main__:--> Novo Registro: ['__import__.clv_patient_03673151817' 'Carlos Eduardo Vercelino'
 '036.731.518-17' Timestamp('1956-08-28 00:00:00') 'Male' 'JCAFB-2025'
 '13026-137' 'Av. Princesa Doeste' 781 'Apto. 41' 'Jardim Proenca'
 'Campinas' '' 'São Paulo (BR)' '' 'Brasil' '' '(19) 9 8191-7639' '' '' ''
 '' '' '' '' '' '' '' '' '' 'x' nan]


### Cadastro de Campanha (arquivo 2)

In [35]:
new_records_inclusion(jcafb_cadastro_campanha_2, 'arquivo_2')


INFO:__main__:--> Novo Registro: ['__import__.clv_patient_15852227802' 'Lamara Guiçardi Vercelino'
 '158.522.278-02' Timestamp('1960-07-06 00:00:00') 'Female' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' 'x' nan]


### Cadastro de Campanha (arquivo 3)

In [36]:
new_records_inclusion(jcafb_cadastro_campanha_3, 'arquivo_3')


INFO:__main__:--> Novo Registro: ['__import__.clv_patient_10649967878' 'Maria Nilde Damico Guicardi'
 '106.499.678-78' Timestamp('1938-03-02 00:00:00') 'Female' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' 'x' nan]


### Cadastro de Campanha (arquivo 4)

In [37]:
new_records_inclusion(jcafb_cadastro_campanha_4, 'arquivo_4')


INFO:__main__:--> Novo Registro: ['__import__.clv_patient_30698626833' 'Moacyr Guicardi' '306.986.268-33'
 Timestamp('1936-03-13 00:00:00') 'Male' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' 'x' nan]


## Lista de Registros incluídos

In [38]:
new_records

,id,name,code,source,source_row,updt_row
0,__import__.clv_patient_03673151817,Carlos Eduardo Vercelino,036.731.518-17,arquivo_1,845,845
1,__import__.clv_patient_15852227802,Lamara Guiçardi Vercelino,158.522.278-02,arquivo_2,845,846
2,__import__.clv_patient_10649967878,Maria Nilde Damico Guicardi,106.499.678-78,arquivo_3,845,847
3,__import__.clv_patient_30698626833,Moacyr Guicardi,306.986.268-33,arquivo_4,845,848


## Cadastro de Pacientes Atualizado

In [39]:
print(jcafb_cadastro_pacientes_updt)

                                     id                         name  \
0    __import__.clv_patient_70905207815           Abel Lucas Bertoni   
1    __import__.clv_patient_82625760800           Adalberto de Sousa   
2    __import__.clv_patient_28019727892   Adelaide Francisco Machado   
3    __import__.clv_patient_05852199826     Adelia Aparecida Fontana   
4    __import__.clv_patient_04227049890       Ademar Antonio Dagoano   
..                                  ...                          ...   
842  __import__.clv_patient_04980187846        Zilda Teixeira Cintra   
843  __import__.clv_patient_03673151817     Carlos Eduardo Vercelino   
844  __import__.clv_patient_15852227802    Lamara Guiçardi Vercelino   
845  __import__.clv_patient_10649967878  Maria Nilde Damico Guicardi   
846  __import__.clv_patient_30698626833              Moacyr Guicardi   

               code   birthday  gender    phase_id        zip  \
0    709.052.078-15 1955-06-01    Male  JCAFB-2025  16680-000   
1    

## Atualizar Registros existentes

In [40]:
column_names = ['id', 'name', 'code', 'column_name', 'from', 'to', 'source', 'source_row']
updt_records = pd.DataFrame(columns=column_names)

def new_records_inclusion(jcafb_cadastro_campanha, source):

    column_names_list = ['name', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2',
                         'city_id', 'city_id/id', 'state_id', 'state_id/id', 'country_id/name', 'country_id/id', 'phone']

    for i, row in jcafb_cadastro_campanha.iterrows():

        _id = row['id']
        index = jcafb_cadastro_pacientes_ref.loc[jcafb_cadastro_pacientes_ref['id'] == _id]
        revised = False
        if len(index) > 0:
            for column_name in column_names_list:
                if row[column_name] != index[column_name].iloc[0]:
                    jcafb_cadastro_pacientes_updt.loc[jcafb_cadastro_pacientes_updt['id'] == _id, column_name] = row[column_name]
                    revised = True

                    new_row = {'id': _id,
                               'name': row['name'],
                               'code': row['code'],
                               'column_name': column_name,
                               'from': index[column_name].iloc[0],
                               'to': row[column_name],
                               'source': source,
                               'source_row': i + 2
                              }
                    pos = len(updt_records)
                    updt_records.loc[pos] = new_row
                    _logger.info(u'%s %s %s', '-->', 'Atualização:', new_row)

            if revised:
                jcafb_cadastro_pacientes_updt.loc[jcafb_cadastro_pacientes_updt['id'] == _id, 'revised'] = 'x'


### Cadastro de Campanha (arquivo 1)

In [41]:
new_records_inclusion(jcafb_cadastro_campanha_1, 'arquivo_1')


INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_70905207815', 'name': 'Abel Lucas Bertoni x', 'code': '709.052.078-15', 'column_name': 'name', 'from': 'Abel Lucas Bertoni', 'to': 'Abel Lucas Bertoni x', 'source': 'arquivo_1', 'source_row': 2}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_70905207815', 'name': 'Abel Lucas Bertoni x', 'code': '709.052.078-15', 'column_name': 'birthday', 'from': Timestamp('1955-06-01 00:00:00'), 'to': Timestamp('1954-06-01 00:00:00'), 'source': 'arquivo_1', 'source_row': 2}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_70905207815', 'name': 'Abel Lucas Bertoni x', 'code': '709.052.078-15', 'column_name': 'gender', 'from': 'Male', 'to': 'Female', 'source': 'arquivo_1', 'source_row': 2}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_70905207815', 'name': 'Abel Lucas Bertoni x', 'code': '709.052.078-15', 'column_name': 'street_name', 'from': 'Rua Coronel Juvencio Silva', 'to': 'Rua Coronel Juvencio

### Cadastro de Campanha (arquivo 2)

In [42]:
new_records_inclusion(jcafb_cadastro_campanha_2, 'arquivo_2')


INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_70905207815', 'name': 'Abel Lucas Bertoni', 'code': '709.052.078-15', 'column_name': 'birthday', 'from': Timestamp('1955-06-01 00:00:00'), 'to': Timestamp('1955-07-01 00:00:00'), 'source': 'arquivo_2', 'source_row': 2}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_82625760800', 'name': 'Adalberto de Sousa x', 'code': '826.257.608-00', 'column_name': 'name', 'from': 'Adalberto de Sousa', 'to': 'Adalberto de Sousa x', 'source': 'arquivo_2', 'source_row': 3}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_82625760800', 'name': 'Adalberto de Sousa x', 'code': '826.257.608-00', 'column_name': 'birthday', 'from': Timestamp('1955-12-12 00:00:00'), 'to': Timestamp('1955-10-12 00:00:00'), 'source': 'arquivo_2', 'source_row': 3}


### Cadastro de Campanha (arquivo 3)

In [43]:
new_records_inclusion(jcafb_cadastro_campanha_3, 'arquivo_3')


INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_28019727892', 'name': 'Adelaide Francisco Machado x', 'code': '280.197.278-92', 'column_name': 'name', 'from': 'Adelaide Francisco Machado', 'to': 'Adelaide Francisco Machado x', 'source': 'arquivo_3', 'source_row': 4}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_28019727892', 'name': 'Adelaide Francisco Machado x', 'code': '280.197.278-92', 'column_name': 'birthday', 'from': Timestamp('1948-03-13 00:00:00'), 'to': Timestamp('1948-03-10 00:00:00'), 'source': 'arquivo_3', 'source_row': 4}


### Cadastro de Campanha (arquivo 4)

In [44]:
new_records_inclusion(jcafb_cadastro_campanha_4, 'arquivo_4')


INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_05852199826', 'name': 'Adelia Aparecida Fontana x', 'code': '058.521.998-26', 'column_name': 'name', 'from': 'Adelia Aparecida Fontana', 'to': 'Adelia Aparecida Fontana x', 'source': 'arquivo_4', 'source_row': 5}
INFO:__main__:--> Atualização: {'id': '__import__.clv_patient_05852199826', 'name': 'Adelia Aparecida Fontana x', 'code': '058.521.998-26', 'column_name': 'birthday', 'from': Timestamp('1964-08-29 00:00:00'), 'to': Timestamp('1960-08-29 00:00:00'), 'source': 'arquivo_4', 'source_row': 5}


## Lista de Atualizações

In [45]:
updt_records

,id,name,code,column_name,from,to,source,source_row
0,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,name,Abel Lucas Bertoni,Abel Lucas Bertoni x,arquivo_1,2
1,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,birthday,1955-06-01 00:00:00,1954-06-01 00:00:00,arquivo_1,2
2,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,gender,Male,Female,arquivo_1,2
3,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,street_name,Rua Coronel Juvencio Silva,Rua Coronel Juvencio Silva xx,arquivo_1,2
4,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,street_number,152,152x,arquivo_1,2
5,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,street_number2,Casa,Casa 3,arquivo_1,2
6,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,street2,Centro,Centro x,arquivo_1,2
7,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,city_id,Avaí,Batatais,arquivo_1,2
8,__import__.clv_patient_70905207815,Abel Lucas Bertoni x,709.052.078-15,phone,(14) 99797-9199,(14) 99797-9155,arquivo_1,2
9,__import__.clv_patient_82625760800,Adalberto de Sousa,826.257.608-00,phase_id,JCAFB-2025,JCAFB-2024,arquivo_1,3


## Cadastro de Pacientes Atualizado

In [46]:
 print(jcafb_cadastro_pacientes_updt)

                                     id                          name  \
0    __import__.clv_patient_70905207815          Abel Lucas Bertoni x   
1    __import__.clv_patient_82625760800          Adalberto de Sousa x   
2    __import__.clv_patient_28019727892  Adelaide Francisco Machado x   
3    __import__.clv_patient_05852199826    Adelia Aparecida Fontana x   
4    __import__.clv_patient_04227049890        Ademar Antonio Dagoano   
..                                  ...                           ...   
842  __import__.clv_patient_04980187846         Zilda Teixeira Cintra   
843  __import__.clv_patient_03673151817      Carlos Eduardo Vercelino   
844  __import__.clv_patient_15852227802     Lamara Guiçardi Vercelino   
845  __import__.clv_patient_10649967878   Maria Nilde Damico Guicardi   
846  __import__.clv_patient_30698626833               Moacyr Guicardi   

               code   birthday  gender    phase_id        zip  \
0    709.052.078-15 1955-07-01  Female  JCAFB-2025  16680-

# Exportar os arquivos

## Exportar Lista de Códigos Inválidos

In [47]:
invalid_codes.to_excel(invalid_codes_path, sheet_name=invalid_codes_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'invalid_codes:', invalid_codes_path)

INFO:__main__:--> invalid_codes: data/jcafb_2025_dhc1_invalid_codes.xlsx


## Exportar Lista de Ids incluídos

In [48]:
new_ids.to_excel(new_ids_path, sheet_name=new_ids_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', new_ids_path)

INFO:__main__:--> new_records: data/jcafb_2025_dhc1_new_ids.xlsx


## Exportar a Lista de Registros incluídos

In [49]:
new_records.to_excel(new_records_path, sheet_name=new_records_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', new_records_path)

INFO:__main__:--> new_records: data/jcafb_2025_dhc1_new_records.xlsx


## Exportar a Lista de Atualizações

In [50]:
updt_records.to_excel(updt_records_path, sheet_name=updt_records_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', updt_records_path)

INFO:__main__:--> new_records: data/jcafb_2025_dhc1_updt_records.xlsx


## Explortar o Cadastro de Pacientes Atualizado

## Exportar o Cadastro de Pacientes para importação pelo Odoo

In [51]:
jcafb_cadastro_pacientes_updt.to_excel(jcafb_cadastro_pacientes_updt_path,
                                       sheet_name=jcafb_cadastro_pacientes_updt_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', jcafb_cadastro_pacientes_updt_path)

INFO:__main__:--> new_records: data/jcafb_2025_cadastro_pacientes_dhc1_updt.xlsx


In [52]:
# jcafb_cadastro_odoo_path = 'data/jcafb_cadastro_odoo.xlsx'
# jcafb_cadastro_odoo_sheet = 'Sheet1'

# # column_names = ['id', 'name', 'code', 'gender', 'birthday', 'street', 'street_number',
# #                 'street_number2', 'street2', 'zip', 'city_id/id', 'state_id/id', 'country_id/id',
# #                 'phone', 'mobile', 'email', 'marker_ids/id', 'phase_id']
# column_names = ['id', 'name', 'code', 'birthday', 'gender', 'phase_id', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 
#                 'city_id/id', 'state_id/id', 'country_id/id', 'phone',
#                 'employee_id/id', 'category_ids/id', 'marker_ids/id', 'tag_ids/id']
# jcafb_cadastro_odoo = pd.DataFrame(columns=column_names)

# for i, row in jcafb_cadastro_pacientes_ref.iterrows():

#     _id = False
#     name = False
#     _code = row['ID']
#     code = False
#     gender = False
#     birthday = False
#     street = False
#     street_number = False
#     street_number2 = False
#     street_number2 = False
#     _zip = False
#     city = False
#     state = False
#     country = False
#     phone = False
#     marker_ids = False
#     phase_id = 'JCAFB-2025'
#     # state = 'Available'

#     sequence_str = ''
#     for c in _code:
#         if c.isdigit():
#             sequence_str = sequence_str + c

#     _id = '__import__.clv_patient_' + sequence_str
#     name = row['Nome']
#     code = _code
#     if row['Sexo'] == 'Masculino':
#         gender = 'Male'
#     if row['Sexo'] == 'Feminino':
#         gender = 'Female'
#     birthday = row['Nascimento']
#     street = row['Logradouro']
#     street_number = row['Número']
#     street_number2 = row['Complemento']
#     street2 = row['Bairro']
#     _zip = '16680-000'
#     city = 'clv_l10n_br_base.city_3504305'
#     state_id = 'base.state_br_sp'
#     country = 'base.br'
#     phone = row['Telefone']
#     if row['Fonte'] == 'MC1_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC1_Criancas'
#     if row['Fonte'] == 'MC1_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC1_Idosos'
#     if row['Fonte'] == 'MC2_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC2_Criancas'
#     if row['Fonte'] == 'MC2_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC2_Idosos'
#     if row['Fonte'] == 'MC3_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC3_Criancas'
#     if row['Fonte'] == 'MC3_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC3_Idosos'
#     if row['Fonte'] == 'MC4_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC4_Criancas'
#     if row['Fonte'] == 'MC4_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC4_Idosos'
#     if row['Fonte'] == 'MC5_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC5_Criancas'
#     if row['Fonte'] == 'MC5_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC5_Idosos'
#     if row['Fonte'] == 'MC6_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC6_Criancas'
#     if row['Fonte'] == 'MC6_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC6_Idosos'
#     if row['Fonte'] == 'MC7_Criancas':
#         marker_ids = '__import__.clv_patient_marker_MC7_Criancas'
#     if row['Fonte'] == 'MC7_Idosos':
#         marker_ids = '__import__.clv_patient_marker_MC7_Idosos'

#     new_row = {'id': _id,
#                'name': name,
#                'code': code,
#                'gender': gender,
#                'birthday': birthday,
#                'street': street,
#                'street_number': street_number,
#                'street_number2': street_number2,
#                'street2': street2,
#                'zip': _zip,
#                'city_id/id': city,
#                'state_id/id': state_id,
#                'country_id/id': country,
#                'phone': phone,
#                'marker_ids/id': marker_ids,
#                'phase_id': phase_id,
#                # 'state': state,
#               }
#     cadastr_odoo.loc[len(cadastr_odoo)] = new_row

# cadastr_odoo.to_excel(jcafb_cadastro_odoo_path, sheet_name=jcafb_cadastro_odoo_sheet, index=False)


# Importar o Cadastro de Pacientes atualizado de Avaí

**Nota**:  <span style='color:green '> Repetir duas vezes o procedimento (confirmar) </span>

1. Conectar-se, via *browser*, ao *Odoo* do servidor [clvheatlh-jcafb-2025n-aws-pro](https://34.224.24.169).

1. Acessar a *View* *Patients*:
    - Menu de acesso:
        - **Health** » **Health** » **Patient** » **Patients**

1. Executar a importação do arquivo "**/opt/clvsol/clvhealth_jcafb_2025_15/project/data/JCAFB-2025 - Cadastro de Pacientes - Campanhas 1.xlsx**":

    - Menu de acesso:
        - **Favoritos** » **Importar registros**

    1. Utilizar o botão **Upload File**.

    1. Selecionar o arquivo "**/opt/clvsol/clvhealth_jcafb_2025_15/project/data/JCAFB-2025 - Cadastro de Pacientes - Campanhas 1.xlsx**".

    1. Utilize o botão **Importar** para executar a Ação.

